input data (title, text, date)

In [1]:
import csv
import sys
import os

title = "'Enemy of mankind': Coronavirus deaths top SARS as China returns to work"
text = "SHANGHAI/BEIJING (Reuters) - China raised the death toll from its coronavirus outbreak to 811 on Sunday, passing the number killed globally by the SARS epidemic, as authorities made plans for millions of people returning to work after an extended Lunar New Year break. Many of China’s usually teeming cities have almost become ghost towns during the past two weeks as Communist Party rulers ordered virtual lockdowns, canceled flights, closed factories and shut schools. Even on Monday, a large number of workplaces and schools will remain closed and many white-collar employees will work from home. The scale of the potential hit to an economy that has been the engine of global growth in recent years has taken a toll on financial markets, as shares slumped and investors switched into safe-havens such as gold, bonds and the Japanese yen."
date = 0

make vectors

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import asarray
from numpy import load
import pickle

mainDir = os.path.dirname(os.path.abspath(""))
vectorizerFILE = os.path.join(mainDir, "Practicas\\NN\\vectorizerOBJ.sav")   
vectorizer = pickle.load(open(vectorizerFILE, "rb"))
vectors = vectorizer.transform([text])
adapted_vectors = vectors.toarray()

use NN (without clusters because they don't improve the prediction)

In [3]:
NN = pickle.load(open(os.path.join(os.path.abspath(""), "NN\\NNPickles.sav"), 'rb'))
result = NN.predict(adapted_vectors)
print(result) #

[1]


use date to check if the news is too old or recent:

In [ ]:
if date>0: #sveza vest poredi sa today
    print("moguce da je vest sveza i ne postoje rezultati na guglu")

In [ ]:
if date<-3: #vest starija od baze (npr prvi put kad se pusti skripta za skrepovanje svega, ucita se datum)
    print("stara vest")

use title to see google search results

In [5]:
import Google

search_results = Google.google_API(title)
if search_results is not None:
    if search_results < 200:
        result = result - 0.2
    elif search_results < 200000:
        result = result - 0.15
    elif search_results < 200000000:
        result = result + 0.2
    if result < 0:
        result = 0
    if result > 1:
        result = 1
print(result)

ModuleNotFoundError: No module named 'Google'

### now you can use clusters to make a pretty graph

In [7]:
import pickle
from numpy import load
from sklearn.cluster import KMeans

clusterer = pickle.load(open(os.path.join(os.path.abspath(""), "NN\\clusterPickles.sav"), 'rb'))
clusterer.predict(adapted_vectors)
reliables_per_class = load(os.path.join(os.path.abspath(""), 'NN\\reliables_per_class.npy'))
k_labels = clusterer.labels_

In [ ]:
print(type(clusterer.cluster_centers_))

In [ ]:
print(type(adapted_vectors))

clusterer.cluster_centers_ // SVI prema adapted_vectors ??

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
cosines = cosine_similarity(clusterer.cluster_centers_, adapted_vectors)


In [10]:
print(len(cosines))

100


In [12]:
mainDir = os.path.abspath("")#the main folder which contains other folders
unreliablePath = os.path.join(mainDir, "unreliableCLEAN.csv")
#the same for reliable
reliablePath=os.path.join(mainDir, "reliableALLmost.csv")
print(unreliablePath, reliablePath)

C:\Users\MarijaStankovic\practica externa git\Practicas\unreliableCLEAN.csv C:\Users\MarijaStankovic\practica externa git\Practicas\reliableALLmost.csv


In [13]:
documents = []
database = []
unreliableNUM=0
reliableNUM=0
y= []

with open(unreliablePath, 'r', encoding='utf8') as db:
    reader=csv.reader(db)
    next(reader, None) # skip the first line
    for row in reader:
        documents.append(row[4])
        database.append(row)
unreliableNUM = len(documents)

with open(reliablePath, 'r', encoding='utf8') as db:
    reader=csv.reader(db)
    next(reader, None) # skip the first line
    for row in reader:
        documents.append(row[4])
        database.append(row)
reliableNUM = len(documents) - unreliableNUM

old_vectors = vectorizer.transform(documents)
adapted_old_vectors = old_vectors.toarray()


In [14]:
print(adapted_old_vectors.shape)

(3288, 6705)


In [34]:
all_cosines = cosine_similarity(adapted_old_vectors, adapted_vectors)
array = []
for c in all_cosines:
    array.append(c[0])
print(len(array))

3288


In [41]:
best_matches = []
for i in range(0,5):
    maximal = max(array)
    index = array.index(maximal)
    best_matches.append([index, maximal])
    array[index] = 0
print(best_matches)

[[384, 0.32785760464106767], [54, 0.31073602097752523], [300, 0.30992937326189185], [343, 0.3096718265084433], [2296, 0.3083803230789782]]


In [50]:
src = {}
src["title"] = []
src["date"] = []
src["source"] = []
src["cosinus"] = []
for m in best_matches:
    print(database[m[0]][3])
    src["title"].append(database[m[0]][1])
    src["date"].append(database[m[0]][2])
    src["source"].append(database[m[0]][3])
    src["cosinus"].append(m[1])


https://www.thesun.co.uk
https://www.telegraph.co.uk
https://www.telegraph.co.uk
https://www.telegraph.co.uk



In [51]:
print(src)

{'title': ['Coronavirus: British Airways halts ALL flights to China over killer bug outbreak', "\nIs coronavirus really the next 'Big One'? How worried we should be about the outbreak\n", '\nHong Kong to close borders with mainland China as global alarm spreads over coronavirus\n', '\nCoronavirus: Inside the locked down Chinese city with virus deja vu\n', 'https://www.nytimes.com'], 'date': ['2020-01-29 11:00:00', '2020-01-25 16:00:00+00:00', '2020-01-28 13:22:00+00:00', '2020-01-29 19:00:00+00:00', 'China’s Xi Praises Free Trade. Striking Deals Is Another Matter.'], 'source': ['https://www.thesun.co.uk', 'https://www.telegraph.co.uk', 'https://www.telegraph.co.uk', 'https://www.telegraph.co.uk', ''], 'cosinus': [0.32785760464106767, 0.31073602097752523, 0.30992937326189185, 0.3096718265084433, 0.3083803230789782]}


In [58]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource

# output_file("bar_basic.html")

source = ColumnDataSource(data=dict(x=[], top=[], date=[], source=[]))
source.data = dict(
        x=[2, 4, 6, 8, 10],
        title = src["title"],
        top = src["cosinus"],
        date = src["date"],
        source = src["source"],
    )
TOOLTIPS=[
    ("Title", "@title"),
    ("Date", "@date"),
    ("Source", "@source"),
]


p = figure( plot_height=350, title="Fruit Counts",
           tooltips=TOOLTIPS) #, tools=""  toolbar_location=None x_range=src["title"],
p.vbar(x="x", top="top", width=0.9, line_color="#888888", fill_color="#f55933", source=source)

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

In [66]:
import json
from bokeh.embed import json_item

json.dumps(json_item(p, "graph"))

'{"target_id": "graph", "root_id": "1669", "doc": {"roots": {"references": [{"attributes": {}, "id": "1691", "type": "WheelZoomTool"}, {"attributes": {"fill_alpha": {"value": 0.1}, "fill_color": {"value": "#1f77b4"}, "line_alpha": {"value": 0.1}, "line_color": {"value": "#1f77b4"}, "top": {"field": "top"}, "width": {"value": 0.9}, "x": {"field": "x"}}, "id": "1707", "type": "VBar"}, {"attributes": {"dimension": 1, "ticker": {"id": "1686", "type": "BasicTicker"}}, "id": "1689", "type": "Grid"}, {"attributes": {"callback": null, "tooltips": [["Title", "@title"], ["Date", "@date"], ["Source", "@source"]]}, "id": "1696", "type": "HoverTool"}, {"attributes": {"active_drag": "auto", "active_inspect": "auto", "active_multi": null, "active_scroll": "auto", "active_tap": "auto", "tools": [{"id": "1690", "type": "PanTool"}, {"id": "1691", "type": "WheelZoomTool"}, {"id": "1692", "type": "BoxZoomTool"}, {"id": "1693", "type": "SaveTool"}, {"id": "1694", "type": "ResetTool"}, {"id": "1695", "type"